In [1]:
import numpy as np 
import pandas as pd
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional, Conv1D, GRU
from sklearn.model_selection import train_test_split, StratifiedKFold
from keras.metrics import categorical_accuracy
from keras.regularizers import l1_l2, l2
from keras import backend as K
import tensorflow as tf

C:\Users\anshk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [37]:
# configure
num_encoder_tokens = 25
num_decoder_tokens = 1000
latent_dim = 256

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, None, 25)     0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, None, 1000)   0                                            
__________________________________________________________________________________________________
lstm_13 (LSTM)                  [(None, 256), (None, 288768      input_13[0][0]                   
__________________________________________________________________________________________________
lstm_14 (LSTM)                  [(None, None, 256),  1287168     input_14[0][0]                   
                                                                 lstm_13[0][1]                    
          

In [25]:
import preprocess

train_event_sentences = pd.read_csv('data/2017/2017_labeled_events_sentences.csv')
train_articles = []
train_events = []
for index, row in train_event_sentences.iterrows():
    train_events.append(preprocess.str_to_tup(row['event']))
    train_articles.append('START ' + row['sentence'])
    

In [21]:
tokenizer_decoder = Tokenizer(char_level = False, num_words=5000)
tokenizer_decoder.fit_on_texts(train_articles)

train_output_data = tokenizer_decoder.texts_to_sequences(train_articles)
print(len(train_articles[0])
print(len(train_output_data[0]))

142
146


In [31]:
import re
article_inp = []
article_out = []
for article in train_articles:
    article = article.replace(',', ' ,').replace('.', ' . ')
    tokens = article.split(' ')
    inp = tokens[0]
    out =  tokens[0] + ' ' + tokens[1]
    article_inp.append(inp)
    article_out.append(out)
    for i in range(1, len(tokens) - 1):
        inp += ' ' + tokens[i]
        out += ' ' + tokens[i + 1]
        article_inp.append(inp)
        article_out.append(out)
        
print(article_inp[7:13])
print(article_out[7:13])

['START Boston rookie Jayson Tatum had 14 points', 'START Boston rookie Jayson Tatum had 14 points and', 'START Boston rookie Jayson Tatum had 14 points and 10', 'START Boston rookie Jayson Tatum had 14 points and 10 rebounds', 'START Boston rookie Jayson Tatum had 14 points and 10 rebounds .', 'START Boston rookie Jayson Tatum had 14 points and 10 rebounds . Hayward']
['START Boston rookie Jayson Tatum had 14 points and', 'START Boston rookie Jayson Tatum had 14 points and 10', 'START Boston rookie Jayson Tatum had 14 points and 10 rebounds', 'START Boston rookie Jayson Tatum had 14 points and 10 rebounds .', 'START Boston rookie Jayson Tatum had 14 points and 10 rebounds . Hayward', 'START Boston rookie Jayson Tatum had 14 points and 10 rebounds . Hayward ,']


In [32]:
tokenizer_decoder = Tokenizer(char_level = False, num_words=5000)
tokenizer_decoder.fit_on_texts(train_articles)
decoder_input_data = tokenizer_decoder.texts_to_sequences(article_inp)
decoder_output_data = tokenizer_decoder.texts_to_sequences(article_out)

In [34]:
print(len(decoder_input_data[10]))
print(len(article_inp[10].split()))

11
11
